## Busqueda con retroceso

ouverture du fichier

In [97]:
def open_file(path):
    test=[]
    
    with open(path, 'r') as f:
        lignes = f.readlines()
        for i in range (0, len(lignes), 2):
            m,n = lignes[i].split()
            taille = [int(m),int(n)]
             
            
            figures = lignes[i+1].split()
            liste = [taille]
            for i in range (0,len(figures), 2):
                coordonnees = [figures[i], figures[i+1]]
                liste.append(coordonnees)
        
            test.append(liste)
            
     
            
        return test

In [99]:
donnees = open_file("C:/Users/combl/Documents/Espagne/Algoritmo_basico/practicas/p3/test.txt")

['3 4\n', '0 0 1 1 1 2\n', '3 6\n', '2 1 2 4 0 4\n', '4 3\n', '2 0 3 2 0 2\n', '6 6\n', '3 1 5 4 2 4']


fonction pour determiner les points auxquels il y a un check point

In [150]:
def checkpoints(taille):
    etapes = []
    for i in range(1, 4,1):
        etapes.append( i*taille[0]*taille[1]//4)
    return etapes
        

conditions à remplir:
on ne doit pas retourner là où on était deja
il faut arriver pile au bon moment sur le checkpoint
il faut verifier au point si c'est possible d'y arriver de là où on est
il faut verifier qu'il n'y ait pas de zonnes auquels on ne peut pas accéder

In [372]:
import numpy as np

Création de la première matrice avec une bordure de 1 et un centre de 0 

In [375]:
def new_matrice(taille):
    matrice = np.empty((taille[0]+2,taille[1]+2))
    for i in range (0, taille[0]+2, 1):
        for j in range(0,taille[1]+2,1):
            if (i ==0 or j == 0 or i== taille[0]+1 or j== taille[1]+1) or (i==1 and j==1):
                matrice[i][j]=1
            else :
                matrice[i][j] = 0
    return matrice
        

In [377]:
def verif_manhatan_dist (coord, point, etape, etape_point):
    dist = abs (coord[0] - int( point[0])) + abs(coord[1] - int( point[1]))
    print("dist", dist)
    print("nb_etapes-restantes", int(etape_point)-etape)
    if dist > (int(etape_point)-etape):
        return False
    else:
        return True
    

In [379]:
def verification_entoure(matrice):
    ##revoir la solution finale
    for i in range (1, matrice.shape[0]-1,1):
        for j in range(1, matrice.shape[1]-1):
            voisins = [(1,-1),(1,0),(1,1),(0,-1),(0,0),(0,1),(-1,-1),(-1,0),(-1,1)]
            compteur = 0
            for x in range(0, len(voisins)):
                case_x = voisins[x][0]+i
                case_y = voisins[x][1]+j
                if matrice[case_x, case_y] == 1:
                    compteur += 1
            if compteur == 6:
                return False
    return True
    
            
            

In [381]:
def creation_matrice_marquee(matrice, coor):
    new_matrice = np.empty((matrice.shape[0],matrice.shape[1]))
    for i in range (0,matrice.shape[0],1):
        for j in range (0,matrice.shape[1],1):
            if (i == coor[0] and j == coor[1]) :
                new_matrice[i][j]= 1
            else:
                new_matrice[i][j] = matrice[i][j]
    return new_matrice
            

In [401]:
def parcourir_voisins(coordonnees, liste_solution, etape, checkpoint, etape_actuelle):
    voisins = [(1,-1),(1,0),(1,1),(0,-1),(0,0),(0,1),(-1,-1),(-1,0),(-1,1)]
    matrice = liste_solution[len(liste_solutions)-1]
    print("matrice_au_debut", matrice)
   

    #voir comment mettre à jour
    
    
    for i in range(0, len(voisins)):
        case_x = voisins[i][0]+coordonnees[0]
        print("x:",case_x)
       
        case_y = voisins[i][1]+coordonnees[1]
        print("y:",case_y)
        if matrice[case_x][case_y]==0:
            if (case_x == int(etape[0]) and case_y == int(etape[1])):
                if etape_actuelle == checkpoint:
                    print("je suis au checkpoint")
                    new_matrice = creation_matrice_marquee(matrice,(case_x,case_y))
                    print(new_matrice)
                    
                    if verification_entoure(new_matrice):
                        print("il n'y a pas de point bloqué")
                    else:
                        print("je suis bloqué")
                        
                else:
                    print("je suis trop tot au checkpoint")
                    
            else:
                if verif_manhatan_dist ((case_x,case_y), etape, etape_actuelle, checkpoint):
                    new_matrice = creation_matrice_marquee(matrice,(case_x,case_y))
                    print(new_matrice)
                    print("c'est encore possible pour moi")
                    liste_solution.append((case_x,case_y))
                    if verification_entoure(matrice):
                        print("il n'y a pas de point bloqué")
                    else:
                        print("je suis bloqué")
                        
                else:
                    print("ce n'est plus possible pour moi")
                    
           
        else:
            print("la case est déjà parcourue")
            
    return 0
      
              
    
    

In [403]:
def programme_final(chemin):
    ##inverser les lignes à la fin!!
    
    donnees = open_file(chemin)
    print("donnees:" ,donnees)

    ##creer des listes et boucler après
    taille = donnees[1][0] #meme jeux que le sujet pour l'instant
    print("taille:",taille)
    premier_jeu = donnees[1] #meme jeux que le sujet pour l'instant
    premier_jeu_etapes = premier_jeu[1:len(premier_jeu)]
    ##regler le porobleme du fait qu'on doit ajouter 1 partout
        
    print (premier_jeu)
    premier_jeu_checkpoint = checkpoints(taille)
    print("checkpoints:",premier_jeu_checkpoint)

    matrice = new_matrice(taille)
    etapes = premier_jeu_etapes
    checkpoint = premier_jeu_checkpoint
    ##initialisation 
    liste_solutions = [[(1,1),matrice]]
    prochaine_etape = etapes[0]
    prochain_checkpoint = checkpoint[0]
    etape_actuelle = 0

    parcourir_voisins((1,1), liste_solutions[0], prochaine_etape, prochain_checkpoint,  etape_actuelle)
    
   
    

    
    
    

In [405]:
programme_final("C:/Users/combl/Documents/Espagne/Algoritmo_basico/practicas/p3/test.txt")

['3 4\n', '0 0 1 1 1 2\n', '3 6\n', '2 1 2 4 0 4\n', '4 3\n', '2 0 3 2 0 2\n', '6 6\n', '3 1 5 4 2 4']
donnees: [[[3, 4], ['0', '0'], ['1', '1'], ['1', '2']], [[3, 6], ['2', '1'], ['2', '4'], ['0', '4']], [[4, 3], ['2', '0'], ['3', '2'], ['0', '2']], [[6, 6], ['3', '1'], ['5', '4'], ['2', '4']]]
taille: [3, 6]
[[3, 6], ['2', '1'], ['2', '4'], ['0', '4']]
checkpoints: [4, 9, 13]
matrice_au_debut [[1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]]
x: 2
y: 0
la case est déjà parcourue
x: 2
y: 1
je suis trop tot au checkpoint
x: 2
y: 2
dist 1
nb_etapes-restantes 4
[[1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 0. 0. 0. 0. 1.]
 [1. 0. 1. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]]
c'est encore possible pour moi


NameError: name 'liste_solution' is not defined